In [78]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from scipy import spatial
from sklearn.preprocessing import normalize
from scipy import sparse
from pyunlocbox import functions, solvers
from scipy import spatial
from matplotlib import pyplot as plt

In [79]:
dataset = pd.read_csv('Placement_Data_Full_Class.csv',index_col='sl_no')
dataset.head()

,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
sl_no,,,,,,,,,,,,,,
1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN
5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0


In [80]:
# Categorical features
cat_features = [feature for feature in dataset.columns if df[feature].dtype == 'O']
len(cat_features)

8

## Applying One-hot Encoding

In [81]:
from sklearn.preprocessing import OneHotEncoder
X = OneHotEncoder().fit_transform(dataset).toarray()
y.shape

(215, 658)

## Learning graph using the Q1 algorithm

In [82]:
def mapping(n):
    #Total number of edges
    ne = int(n*(n-1)/2)
    r1 = np.zeros((ne, ))
    r2 = np.zeros((ne, ))
    itr = 0
    for i in np.arange(1, n):
        r1[itr: (itr + (n - i))] = i - 1
        r2[itr: (itr + (n - i))] = np.arange(i, n)
        itr = itr + n - i
    row = np.concatenate((r1, r2))
    col = np.concatenate((np.arange(0, ne), np.arange(0, ne)))
    values = np.ones(len(row))
    K = sparse.coo_matrix((values, (row, col)), shape=(n, ne))
    return lambda w: K.dot(w), lambda d: K.transpose().dot(d)


def learngraph(X,alpha=1,beta=1,step=0.5,maxit=1000, rtol=1e-5):
    N = X.shape[0]
    z = spatial.distance.pdist(X,'euclidean') # calculating pairwise distance
    w0 = np.zeros(z.shape)
    K, Kt = mapping(N)
    norm_K = np.sqrt(2 * (N - 1))
    
    # Assemble functions in the objective
    f1 = functions.func()
    f1._eval = lambda w: 2 * np.dot(w, z)
    f1._prox = lambda w, gamma: np.maximum(0, w - (2 * gamma * z))

    f2 = functions.func()
    f2._eval = lambda w: - alpha * np.sum(np.log(np.maximum(
        np.finfo(np.float64).eps, K(w))))
    f2._prox = lambda d, gamma: np.maximum(
        0, 0.5 * (d + np.sqrt(d**2 + (4 * alpha * gamma))))

    f3 = functions.func()
    f3._eval = lambda w: beta * np.sum(w**2)
    f3._grad = lambda w: 2 * beta * w
    lipg = 2 * beta
    
    # Rescale stepsize
    stepsize = step / (1 + lipg + norm_K)

    # Solve problem
    solver = solvers.mlfbf(L=K, Lt=Kt, step=stepsize)
    problem = solvers.solve([f1, f2, f3], x0=w0, solver=solver, maxit=maxit,
                            rtol=rtol)

    # Transform weight matrix from vector form to matrix form
    W = spatial.distance.squareform(problem['sol'])
    W[W<0] = 0
    return W

In [83]:
X_norm = normalize(X, axis=0, norm='max')
beta=5
alpha = 1
W = learngraph(X_norm,beta = beta,alpha = 1)

Solution found after 636 iterations:
    objective function f(sol) = 4.889107e+02
    stopping criterion: RTOL


In [84]:
print(W.shape)
#print(np.around(W))
print(W)

(215, 215)
[[0.00000000e+00 1.94425404e-07 1.61308310e-06 ... 0.00000000e+00
  3.92367779e-02 0.00000000e+00]
 [1.94425404e-07 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.61308310e-06 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  6.18783410e-07 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.92367779e-02 0.00000000e+00 6.18783410e-07 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
